In [4]:
#from mstrio.api.browsing import get_objects_from_quick_search
from mstr_robotics.report import cube
from mstr_robotics.read_out_prj_obj import read_gen
#from mstr_robotics.prepare_AI_data import redis_mstr_json
from mstr_robotics.redis_db import redis_bi_analysis
from mstrio.connection import Connection
from mstr_robotics.mstr_classes import mstr_global
from mstr_robotics.json_compare import JSONComparator,compare_mstr_objects
from mstr_robotics._helper import msic
from mstr_robotics.redis_db import fetch_it_all
import json
import yaml
import pandas as pd


i_read_gen=read_gen()
#i_redis_mstr_json=redis_mstr_json()
i_JSONComparator=JSONComparator()
i_msic=msic()
i_mstr_global=mstr_global()
i_compare_mstr_objects=compare_mstr_objects()

with open('..\\config\\mstr_redis_y.yml', 'r') as openfile:
    mstr_redis_y = yaml.safe_load(openfile)

with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)
conn_params =  user_d["conn_params"]

redis_con_d=mstr_redis_y["redis_env_d"]["mstr_dev"]
project_prefix=mstr_redis_y["project_prefix"]
prefix_map=mstr_redis_y["prefix_map"]



## Connect to Redis & MSTR

In [5]:
conn = Connection(**conn_params)

i_redis_bi_analysis = redis_bi_analysis( 
    host=redis_con_d["host"],
    port=redis_con_d["port"],
    password=redis_con_d["password"],
    username=redis_con_d["username"],
    decode_responses=redis_con_d["decode_responses"]
)
org_project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
comp_project_id="F8AD06364B29E4FFA3AEC9A34E706A47"

cube_display_difference_id="70EDB2364AE577328CB5F283351051CE"
cube_display_difference_name="display_difference"
read_out_cbe_fold_id="9981CB214C79D75EC342A781539A4E5A"

org_prefix=project_prefix[org_project_id]
comp_prefix=project_prefix[comp_project_id]

Connection to Strategy One Intelligence Server has been established.
No project selected.


KeyError: 'F8AD06364B29E4FFA3AEC9A34E706A47'

## define comparison objects 

In [ ]:
check_obj_id_l=[]
check_obj_id_l.append("261EA48946B7C87BA6A114AA2766205D")
#check_obj_id_l.append("8D679D3C11D3E4981000E787EC6DE8A4")



## compare shortcut list

In [ ]:
short_cut_folder_id="B2A671234A1B55E512EC3F83A24A150A"

conn.select_project(comp_project_id)
all_obj_d_l=i_mstr_global.get_obj_from_sh_fold(conn,folder_id=short_cut_folder_id)
check_obj_id_l=[]
for obj in all_obj_d_l:
    check_obj_id_l.append(obj["id"])
check_obj_id_l

In [ ]:
json_diff_d_l=[]
for obj_id in check_obj_id_l:
    comp_d={}

    conn.select_project(org_project_id)
    comp_d["org_j_f"]={"project_id":org_project_id,"object_id":obj_id,"redis_key":i_compare_mstr_objects.bld_redis_key(conn,obj_id,org_prefix)}
    conn.select_project(comp_project_id)
    comp_d["comp_j_f"]={"project_id":comp_project_id,"object_id":obj_id,"redis_key":i_compare_mstr_objects.bld_redis_key(conn,obj_id,comp_prefix)}
    json_diff_d_l.append(comp_d.copy())
org_root_object_id_l=[]
comp_root_object_id_l=[]

recursive_fg=True

for obj in json_diff_d_l:
    #print(obj.keys())
    org_root_object_id_l.append(obj["org_j_f"]["redis_key"])
    comp_root_object_id_l.append(obj["comp_j_f"]["redis_key"])

i_fetch_it_all=fetch_it_all(i_redis_bi_analysis)
all_org_objects_df = pd.DataFrame(i_fetch_it_all.fetch_all_objects_recursively(root_object_l=org_root_object_id_l,recursive_fg=recursive_fg))
all_comp_objects_df = pd.DataFrame(i_fetch_it_all.fetch_all_objects_recursively(root_object_l=comp_root_object_id_l,recursive_fg=recursive_fg))


In [ ]:
diff_d_l=i_compare_mstr_objects.compare_objects( all_org_objects_df, all_comp_objects_df)

i_cube=cube()
conn.select_project("F8AD06364B29E4FFA3AEC9A34E706A47")

df=pd.DataFrame(diff_d_l)
df["no_number_path"] = df["path_list"].apply(i_JSONComparator.remove_bracket_numbers)
for col in df.columns:
    df[col] = df[col].astype('string')

dict_list = df.to_dict('records')
df_1=pd.DataFrame(dict_list)

obj_prp_rel_d_l=[{"df":df_1,"tbl_name":"obj_prp_rel_df", "update_policy":"Replace" }]

i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_prp_rel_d_l,mtdi_id=cube_display_difference_id,
                              cube_name=cube_display_difference_name,folder_id=read_out_cbe_fold_id,force=True)



In [ ]:

df["no_number_path"] = df["path_list"].apply(i_JSONComparator.remove_bracket_numbers)
for col in df.columns:
    df[col] = df[col].astype('string')

dict_list = df.to_dict('records')
df_1=pd.DataFrame(dict_list)

obj_prp_rel_d_l=[{"df":df_1,"tbl_name":"obj_prp_rel_df", "update_policy":"Replace" }]
cube_display_difference_id=None #"0EA37C2B447E10A0EAE8FE83314E21E0"
cube_display_difference_name="display_differencegffgs"
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_prp_rel_d_l,mtdi_id=cube_display_difference_id,
                              cube_name=cube_display_difference_name,folder_id=read_out_cbe_fold_id,force=True)




In [ ]:
df=pd.DataFrame(diff_d_l)
df